In [2]:
%load_ext sql
# %config SqlMagic.utocommit=False # for engines that do not support autommit

In [3]:
%%sql sqlite:///Northwind2016.sqlite

select name from sqlite_master where type = 'table'

Done.


name
dbo.Categories
dbo.Suppliers
dbo.Orders
dbo.OrderDetails
dbo.Shippers
dbo.Products
dbo.CustomerGroupThresholds
dbo.Employees
dbo.sysdiagrams
dbo.Customers


## 20. Categories, and the total products in
each category
For this problem, we’d like to see the total number of
products in each category. Sort the results by the total
number of products, in descending order.

In [4]:
%%sql sqlite:///Northwind2016.sqlite

SELECT cat.CategoryName, COUNT(prod.ProductID) as count
FROM 'dbo.Categories' as cat
join 'dbo.Products' as prod
on prod.CategoryID = cat.CategoryID
GROUP BY 1
ORDER BY 2 DESC

Done.


CategoryName,count
Confections,13
Seafood,12
Condiments,12
Beverages,12
Dairy Products,10
Grains/Cereals,7
Meat/Poultry,6
Produce,5


## 21. Total customers per country/city
In the Customers table, show the total number of
customers per Country and City

In [5]:
%%sql sqlite:///Northwind2016.sqlite

SELECT City, Country, COUNT(*) as 'number_of_customers'
FROM 'dbo.Customers'
GROUP BY City, Country
ORDER BY 3 DESC
LIMIT 10

Done.


City,Country,number_of_customers
London,UK,6
México D.F.,Mexico,5
Sao Paulo,Brazil,4
Buenos Aires,Argentina,3
Madrid,Spain,3
Rio de Janeiro,Brazil,3
Lisboa,Portugal,2
Nantes,France,2
Paris,France,2
Portland,USA,2


## 22. Products that need reordering
What products do we have in our inventory that
should be reordered? For now, just use the fields
UnitsInStock and ReorderLevel, where UnitsInStock
is less than the ReorderLevel, ignoring the fields
UnitsOnOrder and Discontinued.
Order the results by ProductID.

In [6]:
%%sql sqlite:///Northwind2016.sqlite

SELECT CAST(ProductID as int), ProductName, UnitsInStock, ReorderLevel
FROM 'dbo.Products'
WHERE CAST(UnitsInStock as int) < CAST(ReorderLevel as int)
ORDER BY 1 ASC

Done.


CAST(ProductID as int),ProductName,UnitsInStock,ReorderLevel
2,Chang,17,25
3,Aniseed Syrup,13,25
11,Queso Cabrales,22,30
21,Sir Rodney's Scones,3,5
30,Nord-Ost Matjeshering,10,15
31,Gorgonzola Telino,0,20
32,Mascarpone Fabioli,9,25
37,Gravad lax,11,25
43,Ipoh Coffee,17,25
45,Rogede sild,5,15


## 23. Products that need reordering,
continued
Now we need to incorporate these fields—
UnitsInStock, UnitsOnOrder, ReorderLevel,
Discontinued—into our calculation. We’ll define
“products that need reordering” with the following:
UnitsInStock plus UnitsOnOrder are less than
or equal to ReorderLevel
The Discontinued flag is false (0)

In [7]:
%%sql sqlite:///Northwind2016.sqlite

SELECT CAST(ProductID as int), ProductName, UnitsInStock, UnitsOnOrder, ReorderLevel, Discontinued
FROM 'dbo.Products'
WHERE CAST(UnitsInStock as int) + CAST(UnitsOnOrder as int) <= CAST(ReorderLevel as int) 
AND CAST(Discontinued as int) = 0
ORDER BY 1 ASC

Done.


CAST(ProductID as int),ProductName,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
30,Nord-Ost Matjeshering,10,0,15,0
70,Outback Lager,15,10,30,0


## 24. Customer list by region
A salesperson for Northwind is going on a business
trip to visit customers, and would like to see a list of
all customers, sorted by region, alphabetically.
However, he wants the customers with no region
(null in the Region field) to be at the end, instead of
at the top, where you’d normally find the null values.
Within the same region, companies should be sorted
by CustomerID

In [8]:
%%sql sqlite:///Northwind2016.sqlite

SELECT CompanyName, Region, CustomerID, 
CASE WHEN LENGTH(REGION) = 0 THEN 1 ELSE 0 END AS temp_col
FROM 'dbo.Customers'
ORDER BY temp_col, Region, CustomerID
LIMIT 10

Done.


CompanyName,Region,CustomerID,temp_col
Old World Delicatessen,AK,OLDWO,0
Bottom-Dollar Markets,BC,BOTTM,0
Laughing Bacchus Wine Cellars,BC,LAUGB,0
Let's Stop N Shop,CA,LETSS,0
Hungry Owl All-Night Grocers,Co. Cork,HUNGO,0
GROSELLA-Restaurante,DF,GROSR,0
Save-a-lot Markets,ID,SAVEA,0
Island Trading,Isle of Wight,ISLAT,0
LILA-Supermercado,Lara,LILAS,0
The Cracker Box,MT,THECR,0


## 25.            High freight charges
Some of the countries we ship to have very high freight charges. We'd like to investigate some more shipping options for our customers, to be able to offer them lower freight charges. Return the three ship countries with the highest average freight overall, in descending order by average freight.
 

In [9]:
%%sql sqlite:///Northwind2016.sqlite

SELECT ShipCountry, AVG(Freight) AS AVG_Freight
FROM 'dbo.Orders'
GROUP BY ShipCountry
ORDER BY AVG_Freight DESC
LIMIT 3


Done.


ShipCountry,AVG_Freight
Austria,184.78750000000002
Ireland,145.01263157894738
USA,112.87942622950818


## 26. High freight charges - 2015
We're continuing on the question above on high
freight charges. Now, instead of using all the orders
we have, we only want to see orders from the year
2015.

In [10]:
%%sql sqlite:///Northwind2016.sqlite

SELECT ShipCountry, AVG(Freight) AS AVG_Freight
FROM 'dbo.Orders'
WHERE strftime('%Y', OrderDate) = '2015'
GROUP BY ShipCountry
ORDER BY AVG_Freight DESC
LIMIT 3

Done.


ShipCountry,AVG_Freight
Austria,178.3642857142857
Switzerland,117.17749999999998
France,113.99102564102563


## 27. High freight charges with between
Another (incorrect) answer to the problem above is
this:
Select Top 3
ShipCountry
,AverageFreight = avg(freight)
From Orders
Where
OrderDate between '1/1/2015' and '12/31/2015'
Group By ShipCountry
Order By AverageFreight desc;
Notice when you run this, it gives Sweden as the
ShipCountry with the third highest freight charges.
However, this is wrong - it should be France.
What is the OrderID of the order that the (incorrect)
answer above is missing?

In [11]:
%%sql sqlite:///Northwind2016.sqlite

select * from 'dbo.orders' order by OrderDate
LIMIT 3

Done.


OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
10248,VINET,5,2014-07-04 08:00:00,2014-08-01 00:00:00,2014-07-16 00:00:00,3,32.3800,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,,51100,France
10249,TOMSP,6,2014-07-05 04:00:00,2014-08-16 00:00:00,2014-07-10 00:00:00,1,11.6100,Toms Spezialitäten,Luisenstr. 48,Münster,,44087,Germany
10251,VICTE,3,2014-07-08 14:00:00,2014-08-05 00:00:00,2014-07-15 00:00:00,1,41.3400,Victuailles en stock,"2, rue du Commerce",Lyon,,69004,France


## 28. High freight charges - last year
We're continuing to work on high freight charges.
We now want to get the three ship countries with the
highest average freight charges. But instead of
filtering for a particular year, we want to use the last
12 months of order data, using as the end date the last
OrderDate in Orders.

In [12]:
%%sql sqlite:///Northwind2016.sqlite

select ShipCountry, AVG(Freight) AS AVG_Freight
from  'dbo.orders' 
WHERE OrderDate BETWEEN '2015-05-01' and '2016-05-06'
GROUP BY ShipCountry
ORDER BY AVG_Freight DESC
LIMIT 3

Done.


ShipCountry,AVG_Freight
Ireland,200.21
Austria,186.45960000000002
USA,119.243023255814


## 29. Inventory list
We're doing inventory, and need to show information
like the below, for all orders. Sort by OrderID and
Product ID.

In [31]:
%%sql 


select a.employeeid, a.lastname, a.orderid, b.productname, a.quantity
from  
(select a.employeeid, b.lastname, a.orderid, a.productid, a.quantity
from (select orders.orderid, productid, employeeid, order_details.quantity as quantity
from 'dbo.orderdetails' order_details
join 'dbo.orders' orders
on orders.OrderID = order_details.orderid) as a
join 'dbo.employees' as b 
on a.employeeid = b.employeeid) a 
join 'dbo.products'  b
on a.productid = b.productid
order by orderid, a.productid
limit 10

 * sqlite:///Northwind2016.sqlite
Done.


employeeid,lastname,orderid,ProductName,quantity
5,Buchanan,10248,Queso Cabrales,12
5,Buchanan,10248,Singaporean Hokkien Fried Mee,10
5,Buchanan,10248,Mozzarella di Giovanni,5
6,Suyama,10249,Tofu,9
6,Suyama,10249,Manjimup Dried Apples,40
4,Peacock,10250,Jack's New England Clam Chowder,10
4,Peacock,10250,Manjimup Dried Apples,35
4,Peacock,10250,Louisiana Fiery Hot Pepper Sauce,15
3,Leverling,10251,Gustaf's Knäckebröd,6
3,Leverling,10251,Ravioli Angelo,15


## 30. Customers with no orders
There are some customers who have never actually
placed an order. Show these customers.

In [64]:
%%sql

select a.customerid, b.customerid
from 'dbo.customers' a
left outer join 'dbo.orders' b
on a.customerid == b.customerid
where b.customerid is null

 * sqlite:///Northwind2016.sqlite
Done.


CustomerID,CustomerID_1
FISSA,None
PARIS,None


## 31. Customers with no orders for EmployeeID 4
One employee (Margaret Peacock, EmployeeID 4)
has placed the most orders. However, there are some
customers who've never placed an order with her.
Show only those customers who have never placed
an order with her.

In [65]:
%%sql



 * sqlite:///Northwind2016.sqlite


'Connected: @Northwind2016.sqlite'